In [19]:
import torch, torchvision
import torch.nn as nn
import os
from PIL import Image

In [ ]:
#Настройка путей (запускать один раз!)
os.chdir('..')
os.getcwd()

'c:\\Users\\vlad\\Desktop\\BeansAI'

In [24]:
#VGG-like архитектура
class CoffeeCNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 64, (3, 3), padding=1)
    self.act = nn.ReLU()
    self.pool1 = nn.MaxPool2d(2)

    self.conv2 = nn.Conv2d(64, 128, (3, 3), padding=1)
    self.pool2 = nn.MaxPool2d(2)

    self.conv3 = nn.Conv2d(128, 256, (3, 3), padding=1)
    self.pool3 = nn.MaxPool2d(2)

    self.conv4 = nn.Conv2d(256, 512, (3, 3), padding=1)
    self.pool4 = nn.MaxPool2d(2)
    self.adapool = nn.AdaptiveAvgPool2d((6, 6))

    self.flat = nn.Flatten()
    self.drop = nn.Dropout(0.5)
    self.lin1 = nn.Linear(512 * 6 * 6, 512)
    self.lin2 = nn.Linear(512, 128)
    self.lin3 = nn.Linear(128, 4)


  def forward(self, x):
    x = self.pool1(self.act(self.conv1(x)))
    x = self.pool2(self.act(self.conv2(x)))
    x = self.pool3(self.act(self.conv3(x)))
    x = self.pool4(self.act(self.conv4(x)))
    x = self.adapool(x)
    x = self.flat(x)
    x = self.drop(self.act(self.lin1(x)))
    x = self.drop(self.act(self.lin2(x)))
    out = self.lin3(x)
    return out


In [27]:
def prediction_sample():
    #Загрузка весов и классов модели, используем cpu
    params = torch.load("app/models/coffee_model_best.pth", map_location="cpu")
    classes = params['classes']

    #Инициализация модели 
    new_model = CoffeeCNN()
    new_model.load_state_dict(params['model_state_dict'])

    #Трансформации
    T = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)), torchvision.transforms.ToTensor()])

    #Открываем изображение по относительному пути и делаем предобработку
    image_pth = "app/data/dark.png"
    image = Image.open(image_pth)
    image = T(image)

    #Переводим модель в тестовый режим
    new_model.eval()

    #Получаем уверенность модели и предсказанный класс в читаемом виде
    prediction = new_model(image.unsqueeze(0))
    prediction = torch.softmax(prediction, dim=1)
    conf, cls = torch.max(prediction, 1)
    return f"Уверенность - {round(conf.item(), 4)}, класс: {classes[cls.item()]}"

In [28]:
print(prediction_sample())

Уверенность - 0.9893, класс: Dark
